# Neural Network
This notebook applies a simple model to simple data.  
Data = Iris.  
Model = Fully Connected aka Dense aka Multi-Layer Perceptron (MLP).  
Notebook 02: Here, we normalize the data. However, the results are just as bad. Overfits.

In [1]:
import datetime
print(datetime.datetime.now())
from platform import python_version
print('Python',python_version())
import csv
import numpy as np
import pandas as pd
import matplotlib as plt
import random

2022-09-03 11:44:45.038146
Python 3.10.0


In [2]:
from tensorflow import keras
from keras import layers

In [3]:
from sklearn import datasets
iris = datasets.load_iris()
print('shape',iris.data.shape)
print('features',iris.feature_names)
print('labels',iris.target_names)
X = iris.data
y = iris.target
print('X shape',X.shape)
print('y shape',y.shape)
num_instances = X.shape[0]
num_features =  X.shape[1]

shape (150, 4)
features ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
labels ['setosa' 'versicolor' 'virginica']
X shape (150, 4)
y shape (150,)


In [4]:
X

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.4, 3.7, 1.5, 0.2],
       [4.8, 3.4, 1.6, 0.2],
       [4.8, 3. , 1.4, 0.1],
       [4.3, 3. , 1.1, 0.1],
       [5.8, 4. , 1.2, 0.2],
       [5.7, 4.4, 1.5, 0.4],
       [5.4, 3.9, 1.3, 0.4],
       [5.1, 3.5, 1.4, 0.3],
       [5.7, 3.8, 1.7, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [5.4, 3.4, 1.7, 0.2],
       [5.1, 3.7, 1.5, 0.4],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5. , 3.4, 1.6, 0.4],
       [5.2, 3.5, 1.5, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [4.7, 3.2, 1.6, 0.2],
       [4.8, 3.1, 1.6, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.2, 4.1, 1.5, 0.1],
       [5.5, 4.2, 1.4, 0.2],
       [4.9, 3

The Keras Normalization class:
1. Constructor takes axis, mean, var. By default, axis = -1 (the last axis), mean=0, var=1.
1. The adapt() method tunes the scaling parameters to the given data.
1. From then on, the object scales any given data.
1. Importantly, we adapt() to the training data and apply to the test data.

In [5]:
X_train = X[:125]
normalizer = layers.Normalization() 
normalizer.adapt(X_train)
X = normalizer(X)
X

2022-09-03 11:44:51.149113: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


<tf.Tensor: shape=(150, 4), dtype=float32, numpy=
array([[-0.750525  ,  0.9468912 , -1.1683092 , -1.1630348 ],
       [-1.0040804 , -0.14098878, -1.1683092 , -1.1630348 ],
       [-1.2576365 ,  0.29416332, -1.2262383 , -1.1630348 ],
       [-1.3844143 ,  0.07658701, -1.1103804 , -1.1630348 ],
       [-0.8773027 ,  1.164467  , -1.1683092 , -1.1630348 ],
       [-0.37019113,  1.8171953 , -0.9945227 , -0.88532835],
       [-1.3844143 ,  0.7293154 , -1.1683092 , -1.0241815 ],
       [-0.8773027 ,  0.7293154 , -1.1103804 , -1.1630348 ],
       [-1.6379697 , -0.35856456, -1.1683092 , -1.1630348 ],
       [-1.0040804 ,  0.07658701, -1.1103804 , -1.301888  ],
       [-0.37019113,  1.3820432 , -1.1103804 , -1.1630348 ],
       [-1.1308582 ,  0.7293154 , -1.0524516 , -1.1630348 ],
       [-1.1308582 , -0.14098878, -1.1683092 , -1.301888  ],
       [-1.7647475 , -0.14098878, -1.342096  , -1.301888  ],
       [ 0.13692045,  2.0347712 , -1.284167  , -1.1630348 ],
       [ 0.0101421 ,  2.9050753 , -

In [6]:
X_train = X[:125]
y_train = y[:125]
X_test  = X[125:]
y_test  = y[125:]

Keras offers 2 means of building a model.  
1. The [Sequential class](https://keras.io/api/models/sequential/#sequential-class) is simple. It has an add(layer) method. It maintains layers in the order they were added. It allows linear models only. Every layer has one input and one output.
1. The [Functional API](https://keras.io/guides/functional_api/) is complex. It allows DAG models. Layers can be skipped, reused, and multiply connected.

The Dense layer has these constructor options.
1. num units
1. activation. Default=linear. Choices are relu, elu, softplus (smooth relu), softmax, sigmoid, tanh, exponential. There is an API so you can build your own.
1. use_bias. Default=True
1. kernel_initializer. Default="glorot_uniform"
1. regularizers for the bias, kernel, and activity. Default=None.

In [7]:
inputs = keras.Input(shape=(num_features,))
inputs

<KerasTensor: shape=(None, 4) dtype=float32 (created by layer 'input_1')>

In [8]:
dense1 = layers.Dense(64, activation="relu")(inputs)
dense2 = layers.Dense(64, activation="relu")(dense1)
outputs = layers.Dense(3)(dense2)
outputs

<KerasTensor: shape=(None, 3) dtype=float32 (created by layer 'dense_2')>

In [9]:
model = keras.Model(inputs=inputs, outputs=outputs, name="MLP_1")
model.summary()

Model: "MLP_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 4)]               0         
                                                                 
 dense (Dense)               (None, 64)                320       
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dense_2 (Dense)             (None, 3)                 195       
                                                                 
Total params: 4,675
Trainable params: 4,675
Non-trainable params: 0
_________________________________________________________________


Explain the number of trainable parameters:
1. Dense_0: 5 in * 64 out = 320 
1. Dense_1: 64 in * 64 out + 64 bias = 4096 + 64 = 4160
1. Dense_2: 64 in * 3 out + 3 bias = 192 + 3 = 195

In [10]:
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.RMSprop(),
    metrics=["accuracy"],
)

Note on model.fit() and model.evaluate()  
Both methods are written to be customized.

In [11]:
EPOCHS=5
# Re-running this cell continues training! Must re-compile to start over.
history = model.fit(X_train, y_train, batch_size=64, epochs=EPOCHS, validation_split=0.2)

Epoch 1/5
2/2 [==============================] - 1s 182ms/step - loss: 1.0273 - accuracy: 0.3900 - val_loss: 1.0512 - val_accuracy: 0.5600
Epoch 2/5
2/2 [==============================] - 0s 35ms/step - loss: 0.8307 - accuracy: 0.7800 - val_loss: 1.1910 - val_accuracy: 0.0400
Epoch 3/5
2/2 [==============================] - 0s 33ms/step - loss: 0.7111 - accuracy: 0.9700 - val_loss: 1.3185 - val_accuracy: 0.0000e+00
Epoch 4/5
2/2 [==============================] - 0s 34ms/step - loss: 0.6195 - accuracy: 0.9900 - val_loss: 1.4661 - val_accuracy: 0.0000e+00
Epoch 5/5
2/2 [==============================] - 0s 32ms/step - loss: 0.5439 - accuracy: 1.0000 - val_loss: 1.6394 - val_accuracy: 0.0000e+00


In [12]:
test_scores = model.evaluate(X_test, y_test, verbose=2)
print("Test loss:", test_scores[0])
print("Test accuracy:", test_scores[1])

1/1 - 0s - loss: 1.5480 - accuracy: 0.0000e+00 - 17ms/epoch - 17ms/step
Test loss: 1.5479896068572998
Test accuracy: 0.0
